In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [40]:
df = pd.read_csv("/hpc/compgen/users/cchang/Projects/gene_exp_VAE/data/SyNet_Normalized_Expression_Batchcorrected_Labeled_Filtered_Data_Only.csv", index_col=0)
print(df.head())


/tmp/ipykernel_9043/2771396363.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/hpc/compgen/users/cchang/Projects/gene_exp_VAE/data/SyNet_Normalized_Expression_Batchcorrected_Labeled_Filtered_Data_Only.csv", index_col=0)


   GSM107072;Pawitan  GSM107073;Pawitan  GSM107074;Pawitan  GSM107075;Pawitan  \
0           3.629049           5.280801           5.242942           4.587148   
1           4.332263           4.463688           3.707902           3.915789   
2           5.242035           4.169501           4.486855           4.602680   
3           4.884504           3.731322           3.508488           4.501715   
4           4.219822           4.804504           3.966309           3.728912   

   GSM107076;Pawitan  GSM107077;Pawitan  GSM107078;Pawitan  GSM107079;Pawitan  \
0           4.157379           4.954873           4.576248           4.935607   
1           4.222221           4.147906           4.513669           4.028902   
2           4.063071           4.252299           3.543621           5.058134   
3           4.201238           3.440060           4.393966           5.469683   
4           3.943046           4.062714           4.670924           4.612529   

   GSM107080;Pawitan  GSM1

In [41]:
df.shape

(11749, 3970)

In [42]:
df = df.iloc[:-1, :]
print(df)
# Here we have 11748 genes. 

       GSM107072;Pawitan  GSM107073;Pawitan  GSM107074;Pawitan  \
0               3.629049           5.280801           5.242942   
1               4.332263           4.463688           3.707902   
2               5.242035           4.169501           4.486855   
3               4.884504           3.731322           3.508488   
4               4.219822           4.804504           3.966309   
...                  ...                ...                ...   
11743           5.114423           4.500965           4.126470   
11744           4.181805           5.370797           4.975893   
11745           4.164249           3.530899           4.080236   
11746           4.194224           3.458629           4.035481   
11747           4.762019           6.531339           3.983910   

       GSM107075;Pawitan  GSM107076;Pawitan  GSM107077;Pawitan  \
0               4.587148           4.157379           4.954873   
1               3.915789           4.222221           4.147906   
2        

In [43]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1, 1))
df_scaled = scaler.fit_transform(df)
print(df_scaled)
# See https://stackoverflow.com/questions/61032867/how-to-scale-data-between-1-and-1-in-pandas. 

[[-0.60251665  0.30339016  0.28033907 ... -0.37786997 -0.2982342
  -0.42050508]
 [-0.21714469 -0.14626653 -0.56461446 ... -0.38583091 -0.30740355
   0.38552677]
 [ 0.2814234  -0.30815687 -0.1358441  ... -0.43625668  0.23093531
  -0.27111731]
 ...
 [-0.30921923 -0.65957925 -0.35966525 ...  0.21093143  0.05654576
  -0.47280828]
 [-0.29279227 -0.69934924 -0.38430053 ...  0.14698923 -0.32670686
   0.04930219]
 [ 0.01836773  0.99156014 -0.41268744 ...  0.38457623  0.07521055
   0.50098755]]


In [44]:
df_scaled = pd.DataFrame(df_scaled, columns=df.columns.values.tolist())
print(df_scaled)
# We have 11748 genes. 

       GSM107072;Pawitan  GSM107073;Pawitan  GSM107074;Pawitan  \
0              -0.602517           0.303390           0.280339   
1              -0.217145          -0.146267          -0.564614   
2               0.281423          -0.308157          -0.135844   
3               0.085491          -0.549286          -0.674381   
4              -0.278764           0.041284          -0.422376   
...                  ...                ...                ...   
11743           0.211490          -0.125753          -0.334216   
11744          -0.299598           0.352915           0.133344   
11745          -0.309219          -0.659579          -0.359665   
11746          -0.292792          -0.699349          -0.384301   
11747           0.018368           0.991560          -0.412687   

       GSM107075;Pawitan  GSM107076;Pawitan  GSM107077;Pawitan  \
0              -0.086415          -0.322869           0.126771   
1              -0.459489          -0.286967          -0.315434   
2        

In [ ]:
df_scaled.to_csv('/hpc/compgen/users/cchang/Projects/gene_exp_VAE/data/SyNet_Scaled_Batchcorrected_Labeled_Filtered_Data_Only.csv')

In [26]:
# Create new df for batch labels
import string

id_list = []
study_list = []
cvset_list = []

# Take the head of the df
sample_header_list = list(df)


for name in sample_header_list:
    split = name.split(';') 
    id_list.append(split[0])
    study_list.append(split[1])
    if split[1] == 'TCGA':
        cvset_list.append('val')
    elif split[1] == 'Miller' or split[1] == 'Minn':
        cvset_list.append('test')
    else:
        cvset_list.append('train')
        
study_name_list = sorted(set(study_list))
# study_name_list = set(study_list)
batch_label_list = list(string.ascii_uppercase[:len(study_name_list)])
study_name_dict = dict(zip(study_name_list, batch_label_list))

if len(id_list) == len(study_list) == len(cvset_list):
    df_batch_label = pd.DataFrame(list(zip(id_list, study_list, cvset_list)), columns=['sample_id', 'study_name', 'cvset'])

df_batch_label['batch_label'] = df_batch_label['study_name'].apply (lambda x: study_name_dict.get(x))
df_batch_label = df_batch_label[['sample_id', 'study_name', 'batch_label', 'cvset']]

print(df_batch_label)

            sample_id study_name batch_label  cvset
0           GSM107072    Pawitan           F  train
1           GSM107073    Pawitan           F  train
2           GSM107074    Pawitan           F  train
3           GSM107075    Pawitan           F  train
4           GSM107076    Pawitan           F  train
...               ...        ...         ...    ...
3965  TCGA-E2-A108-01       TCGA           I    val
3966  TCGA-BH-A0B2-01       TCGA           I    val
3967  TCGA-AO-A12B-01       TCGA           I    val
3968  TCGA-E2-A109-01       TCGA           I    val
3969  TCGA-AO-A128-01       TCGA           I    val

[3970 rows x 4 columns]


In [34]:
df_batch_label.to_csv('/hpc/compgen/users/cchang/Projects/gene_exp_VAE/data/SyNet_batching_1.csv', header=True, index=False)

In [35]:
pd.read_csv('/hpc/compgen/users/cchang/Projects/gene_exp_VAE/data/SyNet_batching_1.csv')

,sample_id,study_name,batch_label,cvset
0,GSM107072,Pawitan,F,train
1,GSM107073,Pawitan,F,train
2,GSM107074,Pawitan,F,train
3,GSM107075,Pawitan,F,train
4,GSM107076,Pawitan,F,train
...,...,...,...,...
3965,TCGA-E2-A108-01,TCGA,I,val
3966,TCGA-BH-A0B2-01,TCGA,I,val
3967,TCGA-AO-A12B-01,TCGA,I,val
3968,TCGA-E2-A109-01,TCGA,I,val
